# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `os` module provides functions to perform tasks such as file and directory operations, process management, and environment variable manipulation.
* The `json` module allows developers to load, read and write JSON files.
* The `yaml` module allows developers to load, read and write YAML files.
* The `geemap` module allows interactive analysis and visualization of GEE datasets in a Jupyter environment.
* The `datetime` module supplies classes for manipulating dates and times.
* The `tabulate` module allows the user to display data in a table format.
* The `google.colab` module provides access to some of the unique features and functionality of Google Colab.

In [ ]:
!pip install geemap
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00


In [ ]:
import ee
import os
import json
import yaml
import geemap
import ffmpeg
import datetime
import tabulate

from google.colab import drive

***Update the geemap package***

If you encounter errors with this notebook, please remove the comment in the line below to update the geemap package to the latest version from GitHub. Restart the Kernel (Menu -> Kernel -> Restart) to take effect.

In [ ]:
# geemap.update_package()

# **2) Authentication Procedure**

This section provides instructions for setting up the Google Earth Engine Python API on Colab and for setting up Google Drive on Colab. These steps should be performed each time you start/restart/rollback a Colab session.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers, while the `ee.Initialize` function initializes it. After executing the following cell, the user is prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory.

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4WMnAW-wW40eEGY8KomGsJtWLuBdoU4_EqBlTI63_Yw&tc=LFAUdZu5SWQeHJwjkvkRpfoRPE34A0Rxsnx_J5r4XVA&cc=9WRSILgqEvzii2stNJMtsQbRpZPwTDssRs7vFhOoszU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn9koeY_vMqmt_mP8faH3xrrsHU_sFB6rIdmNYmzJDpq0o00Hbcalc

Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


## **2.2) GD**

The `drive.mount` function allows access to specific folders of Google Drive. Granting access to Google Drive allows code running in the notebook to modify files in Google Drive.

**Note:** When using the `Mount Drive` button in the file browser, no authentication codes are required for notebooks edited only by the current user.

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# **3) Functions**

Data Processing

In [ ]:
def mask_clouds(raster, cloudProbability):
  """
  Description:
    Masks clouds in a raster based on the specified cloud probability.

  Arguments:
    raster (ee.Image): The raster containing the cloud mask.
    cloudProbability (int): The threshold probability for cloud masking.

  Returns:
    The cloud-masked raster layer.
  """
  clouds = ee.Image(raster.get("cloud_mask")).select("probability")
  isNotCloud = clouds.lt(cloudProbability)
  return raster.updateMask(isNotCloud)


def mask_edges(raster):
  """
  Description:
    Masks edges of a raster using masks from different bands.

  Note:
    The masks for the 10m bands sometimes do not exclude bad data at scene edges,
    so we apply masks from the 20m and 60m bands as well.

  Arguments:
    raster (ee.Image): The raster to mask.

  Returns:
    The masked raster layer.

  Example:
    raster = ee.Image("COPERNICUS/S2_CLOUD_PROBABILITY/20190301T000239_20190301T000238_T55GDP")
  """
  edgesMask = raster.select("B8A").mask().updateMask(raster.select("B9").mask())
  return raster.updateMask(edgesMask)


def export_tasks_viewer(exportTasksIds, tableFormat: str = "plain"):
  """
  Description:
    Displays a table view which contains useful information about the provided export tasks.

  Notes:
    * Task_Id: The task identifier.
    * Task_State: One of READY, RUNNING, COMPLETED, FAILED, CANCELLED, UNSUBMITTED or UNKNOWN.
    * Task_Type: One of EXPORT_IMAGE, EXPORT_TILES, EXPORT_FEATURES, EXPORT_VIDEO.
    * Task_Attempt: Number of attempts.
    * Task_Description: A human-readable description of the task.
    * Queue_Time: The time that is taken while being in a queue.
    * Execution_Time: The time spent by the servers executing the task.
    * Completion_Time: SUm of queue and execution times.
    * Error_Message: Failure reason. Appears only if state is FAILED. May also include other fields.

  Arguments:
    exportTasksIdsList (list) (mandatory) A list of export task identifiers.
    tableFormat (str) (optional) The table format to use. Defaults to "plain".

  Returns:
    None, displays the export tasks table.
  """
  taskInfo = []
  tableHeaders = [
    "Task_Id", "Task_State", "Task_Type", "Task_Attempt", "Task_Description",
    "Queue_Time", "Execution_Time", "Completion_Time", "Error_Message"
  ]
  tableFormats = tabulate._table_formats.keys()

  if tableFormat not in tableFormats:
    raise ValueError(f"Invalid table format. Choose from: `{tableFormats}`.")

  # Populate taskInfo.
  for exportTaskId in exportTasksIds:

    taskState = ee.data.getTaskStatus(exportTaskId)[0]["state"]
    taskType = ee.data.getTaskStatus(exportTaskId)[0]["task_type"]
    taskDescription = ee.data.getTaskStatus(exportTaskId)[0]["description"]
    startTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["start_timestamp_ms"]/1000.0)
    updateTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["update_timestamp_ms"]/1000.0)
    creationTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["creation_timestamp_ms"]/1000.0)

    queueTime = None
    taskAttempt = None
    executionTime = None
    completionTime = None

    if taskState not in ["READY", "RUNNING"]:
      queueTime = (startTimestamp - creationTimestamp).total_seconds()
      executionTime = (updateTimestamp - startTimestamp).total_seconds()

    if taskState == "COMPLETED":
      taskAttempt = ee.data.getTaskStatus(exportTaskId)[0]["attempt"]
      completionTime = (updateTimestamp - creationTimestamp).total_seconds()

    try:
      errorMessage = ee.data.getTaskStatus(exportTaskId)[0]["error_message"]
    except KeyError:
      errorMessage = None  # This just means that the export task has not failed.

    taskInfo.append([exportTaskId, taskState, taskType, taskAttempt, taskDescription, queueTime, executionTime, completionTime, errorMessage])

  # Table display.
  table = tabulate.tabulate(taskInfo, headers=tableHeaders, tablefmt=tableFormat)
  print(table)

# **4) Parameters**

In [ ]:
# `Harmonized Sentinel-2 MSI Level-2A`
sen2StartDate = "2023-01-01"
sen2EndDate = "2023-09-01"
sen2CloudProbability = 40

# `Dynamic World Land Cover`
dwStartDate = "2023-01-01"
dwEndDate = "2023-09-01"

# EMS case of interest
caseCode = "emsr692"
caseArea = "magnesia"

# Projection of interest
projectionScale = 10
projectionCRS = "EPSG:4326"

# GEE assets
NON_WATER_SURFACES = [
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/karla/2023_09_07/non_water_surfaces"),
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/magnesia/non_water_surfaces")
]

EVENT_SURFACES = [
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/karditsa/2023_09_07/flood_surfaces"),
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/karla/2023_09_07/flood_surfaces")
]

WATER_SURFACES = [
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/karditsa/2023_09_07/water_surfaces"),
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/karla/2023_09_07/water_surfaces"),
  ee.FeatureCollection("users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/vectors/magnesia/water_surfaces")
]

destinationPath = "users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/rasters/magnesia/2023_09_07/ground_truth"

# GD paths
configFile = "/content/gdrive/MyDrive/t-h-e-s-i-s/configurations/case_studies.json"
# configFile = "/content/gdrive/MyDrive/t-h-e-s-i-s/configurations/case_studies.yaml"

# **5) Configuration**

In [ ]:
# `Dynamic World LULC v1`
dwConfig = {
  "name": "GOOGLE/DYNAMICWORLD/V1",
  "resolution": 10
}

dwLabelsPalette = {
  "water": "419BDF",
  "trees": "397D49",
  "grass": "88B053",
  "flooded vegetation": "7A87C6",
  "crops": "E49635",
  "shrub and scrub": "DFC35A",
  "built": "C4281B",
  "bare": "A59B8F",
  "snow and ice": "B39FE1"
}

dwVisualization = {
  "min": 0,
  "max": 8,
  "bands": ["label"],
  "palette": list(dwLabelsPalette.values())
}

# `Harmonized Sentinel-2 MSI Level-2A`
sen2Config = {
  "name": "COPERNICUS/S2_SR_HARMONIZED"
}

s2CloudsConfig = {
  "name": "COPERNICUS/S2_CLOUD_PROBABILITY"
}

rgbVisualization = {
  "min": 0,
  "max": 3000,
  "bands": ["B4", "B3", "B2"]
}

mndwiVisualization = {
  "min": -1,
  "max": 0.5,
  "bands": ["MNDWI"],
  "palette": ["red", "yellow", "green", "blue"]
}

bandCombinations = {
  "ndwi": {
    "name": "NDWI",
    "expression": "(b('B3') - b('B8')) / (b('B3') + b('B8'))"
  },
  "mndwi": {
    "name": "MNDWI",
    "expression": "(b('B3') - b('B11')) / (b('B3') + b('B11'))"
  }
}

# Classification
classPalette = {
  "non water": "deb887",
  "flood": "C60404",
  "water": "45b6fe"
}

classVisualization = {
  "min": 0,
  "max": 2,
  "palette": list(classPalette.values())
}

In [ ]:
# Parse the appropriate configuration.
extension = os.path.splitext(configFile)[-1]

try:
  with open(configFile, "r") as stream:
    # JSON
    if extension == ".json":
      caseConfigs = json.load(stream)
    # YAML
    elif extension in (".yaml", ".yml"):
      caseConfigs = yaml.safe_load(stream)
    else:
      raise ValueError(f"Unsupported file format `{extension}`. Supported formats: `JSON`, `YAML`")

except FileNotFoundError as e:
  print(f"Error: JSON file not found: {e}")

# GEE assets.
caseConfig = caseConfigs[caseCode][caseArea]
areaOfInterest = ee.FeatureCollection(caseConfig["area_of_interest"])

ring = ee.Geometry.LinearRing(areaOfInterest.geometry().coordinates().flatten())

# **6) Data Processing**

Define the projection of interest.

In [ ]:
projection = ee.Projection(projectionCRS).atScale(projectionScale)

Load, filter and process the raster collections.

In [ ]:
# `Dynamic World Land Cover V1`
dwCollection = ee.ImageCollection(dwConfig["name"])   \
  .filterDate(dwStartDate, dwEndDate)                 \
  .filterBounds(areaOfInterest)

# Generate a mosaic.
dwMosaic = dwCollection   \
  .mode()                 \
  .select(["label"])      \
  .clipToCollection(areaOfInterest)

# `Harmonized Sentinel-2 MSI Level-2A`
s2Collection = ee.ImageCollection(sen2Config["name"])   \
  .filterDate(sen2StartDate, sen2EndDate)               \
  .filterBounds(areaOfInterest)

# Aggregate the system index values.
indexes = s2Collection.aggregate_array("system:index")

# `Sentinel-2: Cloud Probability`
s2Clouds = ee.ImageCollection(s2CloudsConfig["name"])   \
  .filter(ee.Filter.inList("system:index", indexes))

# Join the two collecctions based on the system index values.
joinedS2Collection = ee.Join.saveFirst("cloud_mask").apply(**{
  "primary": s2Collection,
  "secondary": s2Clouds,
  "condition":
    ee.Filter.equals(**{"leftField": "system:index", "rightField": "system:index"})
})

# Perform cloud masking.
joinedS2Collection = ee.ImageCollection(joinedS2Collection)   \
  .map(mask_edges)                                            \
  .map(lambda raster: mask_clouds(raster, sen2CloudProbability))

# Generate a mosaic.
sen2Mosaic = joinedS2Collection       \
  .mean()                             \
  .select(["B11", "B4", "B3", "B2"])  \
  .clipToCollection(areaOfInterest)

# Engineer new bands,

# `MNDWI`
mndwi = sen2Mosaic                                      \
  .expression(bandCombinations["mndwi"]["expression"])  \
  .rename(bandCombinations["mndwi"]["name"])

# Create a constant raster layer per class.
NON_WATER_SURFACES = ee.FeatureCollection(NON_WATER_SURFACES).flatten()
EVENT_SURFACES = ee.FeatureCollection(EVENT_SURFACES).flatten()
WATER_SURFACES = ee.FeatureCollection(WATER_SURFACES).flatten()

background = ee.Image.constant(0).clipToCollection(areaOfInterest).reproject(projection)
mask = ee.Image.constant(1).clipToCollection(areaOfInterest)

# Flooded surfaces.
floodMask = mask.clipToCollection(EVENT_SURFACES)

eventSurfaces = background.expression("const + 1", {"const": background.select("constant")})  \
  .updateMask(floodMask)                                                                      \
  .reproject(projection)

# Permanent water surfaces.
waterMask = mask.clipToCollection(WATER_SURFACES)

waterSurfaces = background.expression("const + 2", {"const": background.select("constant")})  \
  .updateMask(waterMask)                                                                      \
  .reproject(projection)

# Non water surfaces
nonWaterMask = mask.clipToCollection(NON_WATER_SURFACES)
nonWaterSurfaces = background.updateMask(nonWaterMask).reproject(projection)

# Combine all constant layers.
groundTruth = ee.ImageCollection([background, waterSurfaces, nonWaterSurfaces, eventSurfaces])  \
  .map(lambda raster: raster.int())                                                             \
  .mosaic()                                                                                     \
  .rename("class")                                                                              \
  .reproject(projection)

# **7) Console**

In [ ]:
print("*collection sizes*")
print(f"dynamic-world: `{dwCollection.size().getInfo()}`")
print(f"sentinel-2 L2A: `{s2Collection.size().getInfo()}`")


*collection sizes*
dynamic-world: `157`
sentinel-2 L2A: `248`


# **8) Map Visualization**

In [ ]:
Map = geemap.Map()
Map.centerObject(ring)

# Generate categorical legends.

# `Dynamic-World`
Map.add_legend(title="DW", position="bottomright", legend_dict = dwLabelsPalette)

# `Classification-Labels`
Map.add_legend(title="RF", position="topright", legend_dict = classPalette)

Map.addLayer(sen2Mosaic, rgbVisualization, "rasters: S2-L2A (RGB)")
Map.addLayer(mndwi, mndwiVisualization, "rasters: S2-L2A (MNDWI)")
Map.addLayer(dwMosaic, dwVisualization, "rasters: DW")

Map.addLayer(groundTruth, classVisualization, "rasters: ground-truth")

Map.addLayer(ring, {"color": "white"}, "vectors: region")
Map.addLayer(eventSurfaces, {"color": "black"}, "vectors: observed-event")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# **9) Data Export**

In [ ]:
exportTask = ee.batch.Export.image.toAsset(**{
  "image": groundTruth,
  "assetId": destinationPath,
  "description": "_".join([caseArea, "ground_truth"]),
  "maxPixels": 1e13,
  "crs": projectionCRS,
  "scale": projectionScale,
  "region": areaOfInterest.geometry(),
})

# Submit the tasks.
exportTask.start()

In [ ]:
# Monitor the classifier tasks.
export_tasks_viewer([exportTask.id])

Task_Id                   Task_State    Task_Type       Task_Attempt  Task_Description                   Queue_Time    Execution_Time    Completion_Time  Error_Message
67AODJDMDOY5AXQBTWHSMPD7  COMPLETED     EXPORT_IMAGE               1  farkadona_ground_truth_to_asset        14.477           143.944            158.421


-End of Notebook-